In [31]:
!pip install mediacloud

In [32]:
import mediacloud.api
import datetime as dt
import yfinance as yf
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from datetime import datetime, timedelta
import pandas as pd

# Init the api key and load the finBERT model
MC_API_KEY = "fead6119dad96e93df3254ac0447a4f3f7c1cd48"
US_NATIONAL_COLLECTION = 34412234
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone')

In [33]:
# generate a date range between the start and end
def get_date_range(start_date, end_date):
    date_range = []
    current_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')
    while current_date <= end_date:
        date_range.append(current_date.strftime('%Y-%m-%d'))
        current_date += timedelta(days=1)
    return date_range

In [34]:
def split_string(s):
    split_pos = min(
        s.find(' ') if s.find(' ') != -1 else float('inf'),
        s.find(',') if s.find(',') != -1 else float('inf')
    )

    if split_pos == float('inf'):
        return s

    return s[:split_pos]

In [35]:
# Call the newsapi to generate a single day's news title
def news_data(ticker, date):
  mc_search = mediacloud.api.SearchApi(MC_API_KEY)
  all_stories = []
  pagination_token = None
  more_stories = True
  # Get the Corpornate name as the input for the Media Cloud API
  stockInfo = yf.Ticker(ticker).info['shortName']
  stock = split_string(stockInfo)
  inputDate = dt.datetime.strptime(date, "%Y-%m-%d").date()
  while more_stories:
      page, pagination_token = mc_search.story_list(stock, start_date=inputDate, end_date=inputDate, collection_ids=[US_NATIONAL_COLLECTION])
      all_stories += page
      more_stories = pagination_token is not None

  text = ''
  for story in all_stories[:15]:
      if story['title'] is not None:
        text = text + story['title']
  return text

In [36]:
# Use the news title as the input for the finBERT model
def bert_analyze(text):
  inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)

  with torch.no_grad():
      outputs = model(**inputs)

  logits = outputs.logits

  predicted_class = torch.argmax(logits, dim=1).item()
  sentiment = ['neutral', 'positive', 'negative'][predicted_class]

  return sentiment

In [37]:
def news_bert_analyze(tickers, start_date, end_date):
  df = pd.DataFrame()
  date_range = get_date_range(start_date, end_date)
  for ticker in tickers:
    date_dict = {}
    for date in date_range:
      sentiment = bert_analyze(news_data(ticker, date))
      date_dict[date] = str(sentiment)
    df[ticker]=pd.Series(date_dict)

  df.index.name = 'Datetime'
  # Replace the sensiment with the Int number range from -1 to 1
  df.replace({'neutral': 0, 'positive': 1, 'negative': -1}, inplace=True)
  return (df)

In [30]:
tickers = ['GOOG','MSFT','TSLA']
start_date = '2022-06-01'
end_date = '2022-06-15'

news_bert_analyze(tickers=tickers, start_date=start_date, end_date=end_date)

,GOOG,MSFT,TSLA
Datetime,,,
2022-06-01,0,0,0
2022-06-02,0,0,0
2022-06-03,0,0,0
2022-06-04,1,0,0
2022-06-05,0,0,0
2022-06-06,1,0,1
2022-06-07,1,0,-1
2022-06-08,1,0,1
2022-06-09,0,0,-1
